#I) *IMPLEMENTANDO EM ALGORITMOS GENETICOS.*

---



#Bibliotecas.

In [10]:
import numpy as np
import random
from deap import base, creator, tools, algorithms

#Matriz de custos.

In [13]:
matriz_custos = np.array([
    [0, 10, 15, 45, 5, 45, 50, 44, 30, 100, 67, 33, 90, 17, 50],
    [15, 0, 100, 30, 20, 25, 80, 45, 41, 5, 45, 10, 90, 10, 35],
    [40, 80, 0, 90, 70, 33, 100, 70, 30, 23, 80, 60, 47, 33, 25],
    [40, 80, 5, 0, 5, 40, 21, 20, 30, 14, 55, 35, 21, 5, 40],
    [100, 8, 5, 45, 0, 14, 50, 27, 33, 60, 17, 10, 20, 13, 71],
    [15, 70, 90, 20, 11, 0, 15, 35, 30, 15, 18, 35, 15, 90, 23],
    [25, 19, 18, 30, 100, 55, 0, 70, 55, 41, 55, 100, 18, 14, 18],
    [40, 15, 60, 45, 70, 33, 25, 0, 27, 60, 80, 35, 30, 41, 35],
    [21, 34, 17, 10, 11, 40, 8, 32, 0, 47, 76, 40, 21, 90, 21],
    [35, 100, 5, 18, 43, 25, 14, 30, 39, 0, 17, 35, 15, 13, 40],
    [38, 20, 23, 30, 5, 55, 50, 33, 70, 14, 0, 60, 30, 35, 21],
    [15, 14, 45, 21, 100, 10, 8, 20, 35, 43, 8, 0, 15, 100, 23],
    [80, 10, 5, 20, 35, 8, 90, 5, 44, 10, 80, 14, 0, 25, 80],
    [33, 90, 40, 18, 70, 45, 25, 23, 90, 44, 43, 70, 5, 0, 25],
    [25, 70, 45, 50, 5, 45, 20, 100, 25, 50, 35, 10, 90, 5, 0]
])

#Parâmetros do Algoritmo Genetico.


In [17]:
num_populacao = 100
num_geracoes = 1000
probabilidade_mutacao = 0.1
num_cidades = matriz_custos.shape[0]


#Função para calcular o custo total de uma rota.

In [18]:

def calcular_custo(rota):
    custo_total = 0
    for i in range(num_cidades - 1):
        custo_total += matriz_custos[rota[i], rota[i + 1]]
    custo_total += matriz_custos[rota[-1], rota[0]]  #Retorna à cidade inicial
    return custo_total


# Função para gerar uma população inicial.

In [19]:
def gerar_populacao_inicial():
    populacao = []
    for _ in range(num_populacao):
        rota = list(range(num_cidades))
        random.shuffle(rota)  #Embaralha para gerar uma solução aleatória
        populacao.append(rota)
    return populacao

#Função para selecionar pais com base na aptidão.

In [20]:
def selecionar_pais(populacao):
    aptidao = [1 / calcular_custo(rota) for rota in populacao]
    soma_aptidao = sum(aptidao)
    probabilidade = [aptidao[i] / soma_aptidao for i in range(len(aptidao))]

    pais = random.choices(populacao, weights=probabilidade, k=2)
    return pais

# Função de cruzamento (crossover).

In [21]:
def cruzar(pai1, pai2):
    ponto_corte = random.randint(1, num_cidades - 2)
    filho1 = pai1[:ponto_corte] + [cidade for cidade in pai2 if cidade not in pai1[:ponto_corte]]
    filho2 = pai2[:ponto_corte] + [cidade for cidade in pai1 if cidade not in pai2[:ponto_corte]]
    return filho1, filho2

# Função de mutação.

In [22]:
def mutacao(rota):
    if random.random() < probabilidade_mutacao:
        i, j = random.sample(range(num_cidades), 2)
        rota[i], rota[j] = rota[j], rota[i]  #Troca duas cidades
    return rota

# Função principal do algoritmo genético.

In [29]:
def algoritmo_genetico():
    populacao = gerar_populacao_inicial()
    melhor_rota = None
    menor_custo = float('inf')

    for geracao in range(num_geracoes):
        nova_populacao = []
        for _ in range(num_populacao // 2):
            pais = selecionar_pais(populacao)
            filho1, filho2 = cruzar(pais[0], pais[1])
            nova_populacao.append(mutacao(filho1))
            nova_populacao.append(mutacao(filho2))

        #Atualiza a população
        populacao = nova_populacao

        #Avaliar a população e encontrar a melhor rota
        for rota in populacao:
            custo = calcular_custo(rota)
            if custo < menor_custo:
                menor_custo = custo
                melhor_rota = rota

    return melhor_rota, menor_custo

melhor_rota, menor_custo = algoritmo_genetico()

#Exibindo a solução.

In [30]:
print("Melhor rota encontrada:", melhor_rota)
print("Custo total da rota:", menor_custo)

Melhor rota encontrada: [6, 14, 8, 0, 2, 5, 10, 4, 11, 3, 13, 12, 7, 1, 9]
Custo total da rota: 215


#II) *IMPLEMENTANDO EM COLONIA DE FORMIGAS.*

#Parâmetros da Colonia de Formigas.


In [32]:
num_formigas = 50
num_iteracoes = 100
evaporacao = 0.5
alpha = 1  #Peso do feromônio
beta = 2   #Peso da heurística
q0 = 0.9   #Probabilidade de escolha exploratória

# Inicializando feromônios.

In [33]:
num_cidades = matriz_custos.shape[0]
feromônio = np.ones((num_cidades, num_cidades))

# Função para calcular a rota de uma formiga.

In [34]:
def calcular_rota(formiga):
    rota = [formiga[0]]  #Começa na cidade 0
    custo_total = 0

    for i in range(num_cidades - 1):
        cidade_atual = rota[-1]
        cidade_proxima = formiga[i+1]
        custo_total += matriz_custos[cidade_atual, cidade_proxima]

    #Volta para a cidade inicial
    custo_total += matriz_custos[rota[-1], rota[0]]

    return rota, custo_total

 # Função para atualizar feromônio

In [35]:
def atualizar_feromônio(feromônio, rotas, custos):
    #Evaporar o feromônio
    feromônio *= (1 - evaporacao)

    #Adicionar feromônio nas rotas
    for i in range(len(rotas)):
        rota = rotas[i]
        custo = custos[i]
        for j in range(len(rota) - 1):
            cidade_atual = rota[j]
            cidade_proxima = rota[j+1]
            feromônio[cidade_atual, cidade_proxima] += 1 / custo
            feromônio[cidade_proxima, cidade_atual] += 1 / custo  #Matriz simétrica


# Função principal da colonia de formigas

In [36]:
def colonia(matriz_custos):
    melhor_rota = None
    menor_custo = float('inf')

    for _ in range(num_iteracoes):
        rotas = []
        custos = []

        #Construção das rotas das formigas
        for _ in range(num_formigas):
            cidade_inicial = random.randint(0, num_cidades - 1)
            cidade_atual = cidade_inicial
            visitadas = [cidade_atual]
            formiga = [cidade_atual]

            for _ in range(num_cidades - 1):
                probabilidades = np.zeros(num_cidades)
                for cidade in range(num_cidades):
                    if cidade not in visitadas:
                        probabilidades[cidade] = (feromônio[cidade_atual, cidade] ** alpha) * \
                                                 ((1 / matriz_custos[cidade_atual, cidade]) ** beta)

                #Normaliza as probabilidades
                soma_probabilidades = probabilidades.sum()
                probabilidades /= soma_probabilidades
                cidade_proxima = np.random.choice(range(num_cidades), p=probabilidades)
                visitadas.append(cidade_proxima)
                formiga.append(cidade_proxima)
                cidade_atual = cidade_proxima

            #Calculando o custo da rota
            rota, custo = calcular_rota(formiga)
            rotas.append(rota)
            custos.append(custo)

            #Atualizando a melhor solução
            if custo < menor_custo:
                menor_custo = custo
                melhor_rota = rota

        #Atualizando os feromônios
        atualizar_feromônio(feromônio, rotas, custos)

    return melhor_rota, menor_custo

#Rodando o algoritmo colonia de formigas
melhor_rota, menor_custo = colonia(matriz_custos)

# Exibindo a solução.

In [39]:

print("Melhor rota encontrada:", melhor_rota)
print("Custo total da rota:", menor_custo)

Melhor rota encontrada: [3]
Custo total da rota: 411




---



#Relatorio sobre a atividade.


#1. Introdução.
O Problema do Caixeiro Viajante é problema de otimização combinatória, onde um vendedor precisa visitar várias cidades e retornar ao ponto de origem, minimizando a distância total percorrida.

O objetivo deste relatório é ilustrar como esses dois algoritmos buscam uma solução ótima e como os parâmetros de configuração impactam a eficácia dos métodos. Ambos os algoritmos fazem uso de técnicas inspiradas em processos naturais, mas com abordagens e mecanismos de busca diferentes.



---




#2. Metodologia.

#Algoritmo Genético.

*   População inicial: A população de soluções é gerada aleatoriamente.
*   Avaliação da aptidão: O custo de cada rota é calculado.
*   Seleção: A seleção dos pais é feita com base na aptidão das rotas.
*   Cruzamento: O cruzamento mistura duas rotas para gerar filhos.
*   Mutação: A mutação altera aleatoriamente as rotas para explorar novos caminhos.

#Algoritmo de Colônia de Formigas
*    Inicialização de feromônios: Todos os caminhos começam com uma quantidade de feromônio.
*    Exploração: As formigas percorrem as rotas, depositando feromônio nos caminhos percorridos.
*    Evaporação: O feromônio evapora ao longo do tempo, diminuindo a atração por rotas antigas.
*    Atualização dos feromônios: As melhores rotas recebem mais feromônio, guiando as formigas para elas nas próximas iterações.





---




#3. Resultados
Resultados Obtidos:
Algoritmo Genético (AG):

*    Melhor rota encontrada: [6, 14, 8, 0, 2, 5, 10, 4, 11, 3, 13, 12, 7, 1, 9]
*    Custo total: 215
*    O Algoritmo Genético mostrou boa convergência e encontrou uma solução ótima após 1000 gerações.

Algoritmo de Colônia de Formigas :

* Melhor rota encontrada: [3]
*  Custo total: 411
* A colonia de formigas não convergiu adequadamente e encontrou uma solução incompleta e subótima, indicando falhas no processo de exploração.

Evolução e Convergência:

* Algoritmo Genético: Evolui bem, melhorando as soluções a cada geração e convergindo para uma solução de custo baixo.
* Colonia formigas: Teve falhas de convergência e não conseguiu encontrar uma solução válida.

Sensibilidade aos Parâmetros:
* Algoritmo Genético: Menos sensível aos parâmetros, mantendo um bom equilíbrio entre exploração e exploração.
* Colonia formigas: Altamente sensível aos parâmetros, como taxa de evaporação e número de formigas, o que afetou sua capacidade de convergir.